In [1]:
import subprocess 
import json
import os
from dotenv import load_dotenv
from constants import *
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from bit import *
from web3 import Web3
from eth_account import Account

In [2]:
#Nodes runing with POW
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1.8545"))

# Loading EV
load_dotenv("Mnemonic.env")

# Loading Mnemonic EV and (set this mnemonic as an environment variable, and include the one you generated as a fallback using)
mnemonic = os.getenv("Mnemonic")
print(mnemonic)

left choose fashion ill mansion nest try catalog divide sell sniff better media night dust


In [4]:

def derive_wallets(mnemonic, coin, numderive):
    """Use the subprocess library to call the php file script from Python"""
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --numderive="{numderive}" --coin="{coin}" --format=json' 
    
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
   
    keys = json.loads(output)
    return  keys

In [5]:
#Setting dictionary of coins to be used in the wallet

coins = {"eth", "btc-test", "btc"}
numderive = 3

In [6]:
keys = {}
for coin in coins:
    keys[coin]= derive_wallets(os.getenv('mnemonic'), coin, numderive=3)

In [7]:
eth_PrivateKey = keys["eth"][0]['privkey']
btc_PrivateKey = keys['btc-test'][0]['privkey']


print(json.dumps(eth_PrivateKey, indent=4, sort_keys=True))
print(json.dumps(btc_PrivateKey, indent=4, sort_keys=True))

"0xbfec5226cf568d99428a0f78fea5485b42dea3482885495af8108ed41b52a107"
"cTehDfphwRQB2v7YkfRKS7h9mQwRHgYW3T1BmD4nk5msehTVsKZa"


In [8]:
print(json.dumps(keys, indent=4, sort_keys=True))

{
    "btc": [
        {
            "address": "1Fvjt4r2uxSknfuXmCPZkSCit98r7yC9pk",
            "index": 0,
            "path": "m/44'/0'/0'/0/0",
            "privkey": "L5SFMrAHPL9CA2zQTEn5q5PM7RuwAdgGBkG3G3c1u4vzzz7gTJfs",
            "pubkey": "034fe10c93cd710e1eadf33f594a3669cf84e7f12c7083cd83b2c29bdadd34f1c2",
            "pubkeyhash": "a3bb9a0c222b7e28a3372843102b6d2df25df519",
            "xprv": "xprvA2j5qZ8YRiYD4LyRyBAivZQft3hiRHCLpfh1MKwrUwQG535kWcDuwqqJddMazpQHkNvFraaYF3G5AVacNhWUBam9ESRsitfofiNNVH8659N",
            "xpub": "xpub6FiSF4fSG66WGq3u5ChjHhMQS5YCpjvCBtcc9iMU3GwEwqQu49YAVe9nUu5K26eB8t3hekCnk3GqrzK5VF6R4r5fr2VMXFYP48Rsu2bMhpE"
        },
        {
            "address": "1X2tqtETqfQCc2ScWQEWpmHTr255DXRh9",
            "index": 1,
            "path": "m/44'/0'/0'/0/1",
            "privkey": "L1aAiYGvoxMYEW8vZygKoCtVa7UPRwHdEFp5JyGTdwXUwiQBQrEc",
            "pubkey": "03ba064bcabb386cf7a95f11f580983a178efff0e90628ef2762d42e4b2e175127",
            "pubkeyhash": 

In [9]:
def priv_key_to_account(coin, priv_key):
    """Convert the privkey string in a child key to an account object that bit or web3.py can use to transact"""
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
    
eth_acc = priv_key_to_account(ETH,eth_PrivateKey)
btc_acc = priv_key_to_account(BTCTEST,btc_PrivateKey)

In [10]:
print(eth_acc)
print(btc_acc)

<PrivateKeyTestnet: muXUSCD4RY3QyoHPMbDWbCzuwENZjKyPB2>


In [11]:
def create_trx(coin, account, recipient, amount):
    """create the raw, unsigned transaction that contains all metadata needed to transact"""
    global trx_data
    if coin ==ETH:
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        trx_data = {
            "to": recipient,
            "from": account.address,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address)
        }
        return trx_data

    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])

In [12]:
def send_trx(coin, account, recipient, amount):
    """call create_trx, sign the transaction, then send it to the designated network"""
    if coin == "eth": 
        trx_eth = create_trx(coin,account, recipient, amount)
        sign = account.signTransaction(trx_eth)
        result = w3.eth.sendRawTransaction(sign.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        trx_btctest= create_trx(coin,account,recipient,amount)
        sign_trx_btctest = account.sign_transaction(trx_btctest)
        from bit.network import NetworkAPI
        NetworkAPI.broadcast_tx_testnet(sign_trx_btctest)       
        return sign_trx_btctest

In [13]:
# create BTC transaction
create_trx(BTCTEST,btc_acc,"muXUSCD4RY3QyoHPMbDWbCzuwENZjKyPB2", 0.00001)

'{"unspents":[{"amount":100000,"confirmations":1,"script":"76a91499aaebbde36d0b23e589b244ed34b8518638afb388ac","txid":"6efb423d8d91300b187e6a211d5420676ba0ea345d89e3c2f9d57840f42f4204","txindex":0,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295},{"amount":10000,"confirmations":2160,"script":"76a91499aaebbde36d0b23e589b244ed34b8518638afb388ac","txid":"01063416d1770953475b369865b367f6d3c273ed03bc4e1b074358b452b65836","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["muXUSCD4RY3QyoHPMbDWbCzuwENZjKyPB2",1000],["muXUSCD4RY3QyoHPMbDWbCzuwENZjKyPB2",70852]]}'

In [14]:
#send BTC transaction
send_trx(BTCTEST,btc_acc,'muXUSCD4RY3QyoHPMbDWbCzuwENZjKyPB2',0.0001)

'010000000204422ff44078d5f9c2e3895d34eaa06b6720541d216a7e180b30918d3d42fb6e000000006b483045022100ac15d388cfd8d21812afe91164f5f1c17a134d37c66a3b80e3cf3b9d507ce70b02206e5dfcbc2034e08d826265d6bbe3994df12fefca2fffc50017e278fbb1970e84012103286b5153a6c3d1bbb00712b97cde624ceebd516a6997f69bcf0fb5db69393ef1ffffffff3658b652b45843071b4ebc03ed73c2d3f667b36598365b47530977d116340601010000006b48304502210095bb1f53d16f81724660f5b67bc90e01b740fe24c44a769d0ead5fa7fa83e75d022071f94a6869b82468d1e3715debfd0a8b823cf1b7329c0e1bab9bd4e2b80974aa012103286b5153a6c3d1bbb00712b97cde624ceebd516a6997f69bcf0fb5db69393ef1ffffffff0210270000000000001976a91499aaebbde36d0b23e589b244ed34b8518638afb388ac9cf10000000000001976a91499aaebbde36d0b23e589b244ed34b8518638afb388ac00000000'

In [15]:
from web3.middleware import geth_poa_middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
from web3 import Web3, HTTPProvider
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545/0x1ccebf5fb195c753427fdce71a6b37375d789bc4"))
w3.isConnected()

True

In [17]:
w3.eth.getBalance("0x1ccebf5fb195c753427fdce71a6b37375d789bc4")

InvalidAddress: ('Web3.py only accepts checksum addresses. The software that gave you this non-checksum address should be considered unsafe, please file it as a bug on their platform. Try using an ENS name instead. Or, if you must accept lower safety, use Web3.toChecksumAddress(lower_case_address).', '0x1ccebf5fb195c753427fdce71a6b37375d789bc4')

In [ ]:
w3.eth.getBalance("0x9d0Ae1330EbD84Ac9a2055A1AD6C92C459b6a846")